# Project 1
**Finding the Higgs Boson**

## Importing libs and modules

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from implementations import *
from plots import *

### Datasets path

In [2]:
train_path = './data/train.csv'
test_path = './data/test.csv'

## Loading datasets

In [3]:
y, x, ids = load_csv_data(train_path)
ytest, testx, idstest = load_csv_data(test_path)

# Analysis on features

In [4]:
# DERIVATIVE
DER_mass_MMC = 0
DER_mass_transverse_met_lep = 1
DER_mass_vis = 2
DER_pt_h = 3
DER_deltar_tau_lep = 7
DER_pt_tot = 8
DER_sum_pt = 9
DER_pt_ratio_lep_tau = 10
DER_met_phi_centrality = 11
met_phi = 20

# PRIMITIVE
pri_tau_pt = 13
pri_tau_eta = 14
pri_tau_phi = 15
# ---------------
pri_lep_pt = 16
pri_lep_eta = 17
pri_lep_phi = 18

# ------------------------------------problematics--------------------------------------
# DERIVATIVE WITH PROBS
prob_1 = 4
prob_2 = 5
prob_3 = 6
prob_12 = 12
# PRIMITIVE WITH PROBS
pri_jet_pt = 23
pri_jet_eta = 24
pri_jet_phi = 25
pri_sub_pt = 26
pri_sub_eta = 27
pri_sub_phi = 28
pri_all_pt = 29

# Momentum vectors for TAU
px_tau = x[:,pri_tau_pt]*np.sin(x[:,pri_tau_phi])
py_tau = x[:,pri_tau_pt]*np.cos(x[:,pri_tau_phi])
pz_tau = x[:,pri_tau_pt]*np.sinh(x[:,pri_tau_eta])
mod_tau = x[:,pri_tau_pt]*np.cosh(x[:,pri_tau_eta])

# Momentum vectors for LEP
px_lep = x[:,pri_lep_pt]*np.sin(x[:,pri_lep_phi])
py_lep = x[:,pri_lep_pt]*np.cos(x[:,pri_lep_phi])
pz_lep = x[:,pri_lep_pt]*np.sinh(x[:,pri_lep_eta])
mod_lep = x[:,pri_lep_pt]*np.cosh(x[:,pri_lep_eta])

px_taut = testx[:,pri_tau_pt]*np.sin(testx[:,pri_tau_phi])
py_taut = testx[:,pri_tau_pt]*np.cos(testx[:,pri_tau_phi])
pz_taut = testx[:,pri_tau_pt]*np.sinh(testx[:,pri_tau_eta])
mod_taut = testx[:,pri_tau_pt]*np.cosh(testx[:,pri_tau_eta])

px_lept = testx[:,pri_lep_pt]*np.sin(testx[:,pri_lep_phi])
py_lept = testx[:,pri_lep_pt]*np.cos(testx[:,pri_lep_phi])
pz_lept = testx[:,pri_lep_pt]*np.sinh(testx[:,pri_lep_eta])
mod_lept = testx[:,pri_lep_pt]*np.cosh(testx[:,pri_lep_eta])

# Momentum vectors for JET PRIMARY
# px_jet = x[:,pri_jet_pt]*np.sin(x[:,pri_jet_phi])
# py_jet = x[:,pri_jet_pt]*np.cos(x[:,pri_jet_phi])
# pz_jet = x[:,pri_jet_pt]*np.sinh(x[:,pri_jet_eta])
# mod_jet = x[:,pri_jet_pt]*np.cosh(x[:,pri_jet_eta])

# Momentum vectors for JET SECONDARY
# px_sub = x[:,pri_sub_pt]*np.sin(x[:,pri_sub_phi])
# py_sub = x[:,pri_sub_pt]*np.cos(x[:,pri_sub_phi])
# pz_sub = x[:,pri_sub_pt]*np.sinh(x[:,pri_sub_eta])
# mod_sub = x[:,pri_sub_pt]*np.cosh(x[:,pri_sub_eta])

In [5]:
x = np.column_stack((x, px_tau))
x = np.column_stack((x, py_tau))
x = np.column_stack((x, pz_tau))
x = np.column_stack((x, mod_tau))
x = np.column_stack((x, px_lep))
x = np.column_stack((x, py_lep))
x = np.column_stack((x, pz_lep))
x = np.column_stack((x, mod_lep))

testx = np.column_stack((testx, px_taut))
testx = np.column_stack((testx, py_taut))
testx = np.column_stack((testx, pz_taut))
testx = np.column_stack((testx, mod_taut))
testx = np.column_stack((testx, px_lept))
testx = np.column_stack((testx, py_lept))
testx = np.column_stack((testx, pz_lept))
testx = np.column_stack((testx, mod_lept))
testx.shape

(568238, 38)

## Removing column features

In [6]:
columns_feature_to_delete = [pri_jet_phi,pri_sub_phi,pri_tau_phi,pri_lep_phi,met_phi]
x = np.delete(x, columns_feature_to_delete, 1)
testx = np.delete(testx, columns_feature_to_delete, 1)

In [8]:
x.shape

(250000, 33)

In [9]:
x[np.where(x == -999)] = np.nan
me = np.ma.array(x, mask=np.isnan(x)).mean(axis=0)
means = np.ma.getdata(me)
inds = np.where(np.isnan(x))
x[inds]=np.take(means,inds[1])

In [10]:
testx[np.where(testx == -999)] = np.nan
me = np.ma.array(testx, mask=np.isnan(testx)).mean(axis=0)
means = np.ma.getdata(me)
inds = np.where(np.isnan(testx))
testx[inds]=np.take(means,inds[1])

In [11]:
x, mean_x, std_x = standardize(x)
testx, mean_x, std_x = standardize(testx)

[ -1.31421984e-16   4.19362323e-17  -3.43618467e-17   4.41104930e-17
   3.60216745e-16   2.01623607e-16  -5.11306553e-17  -5.14546628e-16
  -4.54633664e-16  -1.03227649e-16   3.73262310e-16  -3.55555585e-17
  -1.05956133e-16   4.05350420e-16   1.33013600e-17  -5.94468474e-16
   1.35571554e-17  -3.88581611e-16  -4.52388349e-16   1.11981535e-17
   2.63526090e-16  -3.41060513e-19   3.53566065e-17  -1.84741111e-19
  -2.37520226e-16   1.77635684e-17  -1.03455022e-17  -2.11173301e-17
   9.23705556e-17   1.75646164e-17   7.61701813e-18   4.66116035e-18
   9.06652531e-18]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ -3.94598691e-16  -2.45834847e-16  -6.11542317e-16   5.75198523e-19
  -3.92110332e-16   5.69971719e-16  -1.45700287e-16   4.30586112e-16
   2.78683685e-16   2.46410046e-16   8.07028537e-17   5.32746371e-17
  -9.01411112e-16   2.67317262e-16  -7.34003333e-18   5.74323221e-17
   2.59902202e-17 

In [13]:
x[2]
testx[2]

array([ -7.79895413e-02,   1.96861734e-01,   3.76430857e-01,
        -8.48157450e-01,  -4.72273126e-16,   5.28757703e-16,
        -1.72252608e-16,   4.88205176e-01,  -6.82689607e-01,
        -5.25281795e-01,  -4.06128940e-01,  -1.07254328e+00,
        -1.03418102e-15,   3.50764965e-01,  -2.35755940e-01,
         1.95227230e-01,  -4.18622522e-01,  -6.85845098e-01,
        -7.75487951e-01,  -1.00087589e+00,  -1.21037878e-15,
        -2.74546365e-17,  -4.03206984e-16,   2.45273670e-17,
        -7.44057124e-01,   1.03962295e-01,  -1.46939356e+00,
        -1.63295725e-01,  -4.25839374e-01,   4.30082002e-01,
         1.30493668e+00,  -2.63486145e-01,  -4.99665669e-01])

# Local test

In [14]:
train_x, train_y, test_x, test_y = split_data(x, y, 0.8, seed=1)

In [11]:
degree = 6
px_train = build_poly(degree=degree,x=train_x)
px_test = build_poly(degree=degree,x=test_x)

In [12]:
px_train.shape

(200000, 199)

# Online test

In [15]:
degree = 6
px_train = build_poly(degree=degree,x=x)
px_test = build_poly(degree=degree,x=testx)

In [19]:
px_test.shape

(568238, 199)

# Logistic regression

In [35]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.0000001
    losses = []

    y[y == -1] = 0
    # build tx
    w = np.zeros((x.shape[1], ))
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, x, w, gamma)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    print("loss={l}".format(l=calculate_loss(y, x, w)))

In [36]:
logistic_regression_gradient_descent_demo(y, x)

Current iteration=0, loss=173286.79513998653
Current iteration=1, loss=172502.18470433177
Current iteration=2, loss=171753.12930779433
Current iteration=3, loss=171037.56370953232
Current iteration=4, loss=170353.5394665347
Current iteration=5, loss=169699.2225117463
Current iteration=6, loss=169072.889762671
Current iteration=7, loss=168472.92498931434
Current iteration=8, loss=167897.81414872222
Current iteration=9, loss=167346.14036730325
Current iteration=10, loss=166816.57872428128
Current iteration=11, loss=166307.8909621195
Current iteration=12, loss=165818.92022407483
Current iteration=13, loss=165348.5858960702
Current iteration=14, loss=164895.87861023037
Current iteration=15, loss=164459.8554507745
Current iteration=16, loss=164039.63538933528
Current iteration=17, loss=163634.3949658796
Current iteration=18, loss=163243.3642228866
Current iteration=19, loss=162865.8228939259
Current iteration=20, loss=162501.09684292568
Current iteration=21, loss=162148.5547469091
Current i

Current iteration=179, loss=147452.049781408
Current iteration=180, loss=147428.39608318746
Current iteration=181, loss=147404.9603124777
Current iteration=182, loss=147381.73915775787
Current iteration=183, loss=147358.72937064458
Current iteration=184, loss=147335.92776447764
Current iteration=185, loss=147313.33121294255
Current iteration=186, loss=147290.9366487286
Current iteration=187, loss=147268.74106222103
Current iteration=188, loss=147246.7415002271
Current iteration=189, loss=147224.9350647342
Current iteration=190, loss=147203.31891169958
Current iteration=191, loss=147181.89024987083
Current iteration=192, loss=147160.6463396358
Current iteration=193, loss=147139.5844919015
Current iteration=194, loss=147118.70206700105
Current iteration=195, loss=147097.9964736278
Current iteration=196, loss=147077.46516779574
Current iteration=197, loss=147057.10565182596
Current iteration=198, loss=147036.9154733577
Current iteration=199, loss=147016.89222438392
Current iteration=200, 

Current iteration=355, loss=145050.38839583477
Current iteration=356, loss=145041.9917722268
Current iteration=357, loss=145033.62602097064
Current iteration=358, loss=145025.29090491947
Current iteration=359, loss=145016.98618965226
Current iteration=360, loss=145008.71164343526
Current iteration=361, loss=145000.46703718373
Current iteration=362, loss=144992.2521444244
Current iteration=363, loss=144984.06674125863
Current iteration=364, loss=144975.91060632607
Current iteration=365, loss=144967.78352076898
Current iteration=366, loss=144959.68526819692
Current iteration=367, loss=144951.61563465244
Current iteration=368, loss=144943.57440857685
Current iteration=369, loss=144935.56138077687
Current iteration=370, loss=144927.57634439162
Current iteration=371, loss=144919.61909486033
Current iteration=372, loss=144911.68942989028
Current iteration=373, loss=144903.78714942568
Current iteration=374, loss=144895.9120556166
Current iteration=375, loss=144888.0639527886
Current iteration

Current iteration=531, loss=143906.5459583105
Current iteration=532, loss=143901.41764600246
Current iteration=533, loss=143896.3006351435
Current iteration=534, loss=143891.19487826023
Current iteration=535, loss=143886.100328214
Current iteration=536, loss=143881.0169381975
Current iteration=537, loss=143875.94466173172
Current iteration=538, loss=143870.88345266265
Current iteration=539, loss=143865.83326515823
Current iteration=540, loss=143860.79405370515
Current iteration=541, loss=143855.7657731059
Current iteration=542, loss=143850.74837847566
Current iteration=543, loss=143845.74182523927
Current iteration=544, loss=143840.74606912842
Current iteration=545, loss=143835.76106617856
Current iteration=546, loss=143830.7867727261
Current iteration=547, loss=143825.8231454056
Current iteration=548, loss=143820.87014114673
Current iteration=549, loss=143815.92771717175
Current iteration=550, loss=143810.99583099264
Current iteration=551, loss=143806.07444040832
Current iteration=552

Current iteration=707, loss=143145.3036426201
Current iteration=708, loss=143141.6433287134
Current iteration=709, loss=143137.98921508
Current iteration=710, loss=143134.34128433053
Current iteration=711, loss=143130.69951915543
Current iteration=712, loss=143127.06390232418
Current iteration=713, loss=143123.434416685
Current iteration=714, loss=143119.8110451641
Current iteration=715, loss=143116.19377076533
Current iteration=716, loss=143112.58257656955
Current iteration=717, loss=143108.9774457342
Current iteration=718, loss=143105.37836149274
Current iteration=719, loss=143101.78530715412
Current iteration=720, loss=143098.19826610244
Current iteration=721, loss=143094.61722179616
Current iteration=722, loss=143091.04215776792
Current iteration=723, loss=143087.4730576239
Current iteration=724, loss=143083.90990504326
Current iteration=725, loss=143080.35268377786
Current iteration=726, loss=143076.8013776516
Current iteration=727, loss=143073.25597056013
Current iteration=728, l

...

Current iteration=9146, loss=139777.45246777794
Current iteration=9147, loss=139777.43804057478
Current iteration=9148, loss=139777.4236192969
Current iteration=9149, loss=139777.40920394147
Current iteration=9150, loss=139777.3947945057
Current iteration=9151, loss=139777.38039098677
Current iteration=9152, loss=139777.3659933819
Current iteration=9153, loss=139777.35160168825
Current iteration=9154, loss=139777.33721590304
Current iteration=9155, loss=139777.3228360235
Current iteration=9156, loss=139777.30846204676
Current iteration=9157, loss=139777.29409397006
Current iteration=9158, loss=139777.27973179062
Current iteration=9159, loss=139777.26537550564
Current iteration=9160, loss=139777.2510251123
Current iteration=9161, loss=139777.23668060784
Current iteration=9162, loss=139777.22234198943
Current iteration=9163, loss=139777.20800925433
Current iteration=9164, loss=139777.19368239972
Current iteration=9165, loss=139777.17936142287
Current iteration=9166, loss=139777.165046320

Current iteration=9318, loss=139775.05583586512
Current iteration=9319, loss=139775.04238769723
Current iteration=9320, loss=139775.02894499243
Current iteration=9321, loss=139775.01550774818
Current iteration=9322, loss=139775.00207596185
Current iteration=9323, loss=139774.98864963092
Current iteration=9324, loss=139774.97522875282
Current iteration=9325, loss=139774.96181332498
Current iteration=9326, loss=139774.94840334484
Current iteration=9327, loss=139774.93499880985
Current iteration=9328, loss=139774.92159971746
Current iteration=9329, loss=139774.90820606507
Current iteration=9330, loss=139774.89481785017
Current iteration=9331, loss=139774.8814350702
Current iteration=9332, loss=139774.86805772258
Current iteration=9333, loss=139774.85468580478
Current iteration=9334, loss=139774.84131931423
Current iteration=9335, loss=139774.8279582484
Current iteration=9336, loss=139774.81460260475
Current iteration=9337, loss=139774.80125238068
Current iteration=9338, loss=139774.787907

Current iteration=9490, loss=139772.82099743764
Current iteration=9491, loss=139772.8084522447
Current iteration=9492, loss=139772.7959120922
Current iteration=9493, loss=139772.78337697778
Current iteration=9494, loss=139772.77084689907
Current iteration=9495, loss=139772.75832185376
Current iteration=9496, loss=139772.74580183948
Current iteration=9497, loss=139772.73328685388
Current iteration=9498, loss=139772.72077689465
Current iteration=9499, loss=139772.70827195942
Current iteration=9500, loss=139772.69577204584
Current iteration=9501, loss=139772.6832771516
Current iteration=9502, loss=139772.67078727434
Current iteration=9503, loss=139772.65830241176
Current iteration=9504, loss=139772.64582256146
Current iteration=9505, loss=139772.63334772116
Current iteration=9506, loss=139772.6208778885
Current iteration=9507, loss=139772.60841306116
Current iteration=9508, loss=139772.5959532368
Current iteration=9509, loss=139772.5834984131
Current iteration=9510, loss=139772.5710485877

Current iteration=9662, loss=139770.73543281693
Current iteration=9663, loss=139770.72372101326
Current iteration=9664, loss=139770.71201386314
Current iteration=9665, loss=139770.7003113645
Current iteration=9666, loss=139770.6886135151
Current iteration=9667, loss=139770.67692031286
Current iteration=9668, loss=139770.66523175559
Current iteration=9669, loss=139770.65354784115
Current iteration=9670, loss=139770.64186856739
Current iteration=9671, loss=139770.6301939322
Current iteration=9672, loss=139770.6185239334
Current iteration=9673, loss=139770.60685856885
Current iteration=9674, loss=139770.59519783643
Current iteration=9675, loss=139770.58354173397
Current iteration=9676, loss=139770.57189025937
Current iteration=9677, loss=139770.56024341047
Current iteration=9678, loss=139770.54860118515
Current iteration=9679, loss=139770.53696358122
Current iteration=9680, loss=139770.5253305966
Current iteration=9681, loss=139770.51370222916
Current iteration=9682, loss=139770.502078476

KeyboardInterrupt: 

# With ridge regression

In [20]:
ws = ridge_regression(lambda_=0.00067233575365, tx=px_train, y=y)

In [21]:
loss = compute_mse(y, px_train, ws)
loss

0.29372310217874009

In [22]:
loss = compute_mse(ytest, px_test, ws)
loss

17.914458797928955

# Prediction

In [23]:
y_pred = predict_labels(ws, px_test)

In [24]:
accuracy = 1 - np.mean( y_pred != test_y )

/Users/Pucci/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [25]:
print("Accuracy: " + str(accuracy) + "%")

Accuracy: 0.0%


# Submission

In [26]:
create_csv_submission(idstest, y_pred, 'prediction.csv')

# Prepare data for k-fold cross validation

In [ ]:
seed = 1
degree = 6
k_fold = 10
lambdas = np.logspace(-4, 0, 30)
k_indices = build_k_indices(y, k_fold, seed)
rmse_tr = []
rmse_te = []
best_loss = 999
for k in range(k_fold):
    temp_tr = []
    temp_te = []
    best_test_ws = []
    for lambda_ in lambdas:
        tr_loss, te_loss, ws = cross_validation(y, x, k_indices, k, lambda_, degree)
        if(te_loss < best_loss):
            best_loss = te_loss
            best_lambda = lambda_
        temp_tr.append(tr_loss)
        temp_te.append(te_loss)
        # print("Lambda = " + str(lambda_) + " tr_loss = " + str(tr_loss) + " te_loss = " + str(te_loss))
    print("After lambdas iteration, the best lambda is : " + str(best_lambda) + " for k-fold : " + str(k) + " with best loss = " + str(best_loss))
    best_test_ws.append(lambda_)
    rmse_tr.append(temp_tr)
    rmse_te.append(temp_te)

rmse_tr = np.matrix(rmse_tr)
rmse_tr = np.mean(rmse_tr, axis=0)
rmse_tr = np.reshape(rmse_tr, (len(lambdas),-1))
rmse_te = np.matrix(rmse_te)
rmse_te = np.mean(rmse_te, axis=0)
rmse_te = np.reshape(rmse_te, (len(lambdas),-1))

cross_validation_visualization(lambdas, rmse_tr, rmse_te)

In [ ]:
def learning_by_newton_method(y, tx, w):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # return loss, gradient and hessian: TODO
    # ***************************************************
    print('here it is')
    loss, gradient, hessian = logistic_regression(y,tx,w)
    
    #raise NotImplementedError
    # ***************************************************
    # INSERT YOUR CODE HERE
    # update w: TODO
    # ***************************************************
    print('here it is')
    termine = np.linalg.solve(hessian,gradient)
    w = w-termine
    
    #raise NotImplementedError
    return loss, w

In [ ]:
def logistic_regression_newton_method_demo(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.01
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((x.shape[1], ))
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, x, w)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_newton_method")
    print("loss={l}".format(l=calculate_loss(y, tx, w)))

In [ ]:
# logistic_regression_newton_method_demo(y,x)